## 1. Set up Environment

In [15]:
import pandas as pd
import requests
import datetime
import pyodbc
from sqlalchemy import create_engine

## 2. Store API Key

In [16]:
key = open('alphavantage_key.txt').read()

## 3. Construct General Form for API Call

In [17]:
class api_construct:
    def __init__(self, function, symbol, apikey):
        self.function = function
        self.symbol = symbol
        self.apikey = apikey
    
    date = "-".join(str(datetime.date.today()).split('-')[:2])

    def intraday(self, interval='1min', adjusted='true', extended_hours='true', month=date, outputsize='compact', datatype='json'):
        self.url = 'https://www.alphavantage.co/query?function=' + self.function + '&symbol=' + self.symbol\
        + '&interval=' + interval + '&adjusted=' + adjusted + '&extended_hours=' + extended_hours + '&month=' + month\
        + '&outputsize=' + outputsize + '&apikey=' + self.apikey + '&datatype=' + datatype
 
# INTRADAY_EXTENDED API has recently (~2023-07-20) been merged with INTRADAY. Keep this here just in case
#    def intraday_ext(self, interval='1min', slice_='year1month1', adjusted='true'):
#        self.url = 'https://www.alphavantage.co/query?function=' + self.function + '&symbol=' + self.symbol\
#        + '&interval=' + interval + '&slice=' + slice_ + '&adjusted=' + adjusted + '&apikey=' + self.apikey

    def daily(self, outputsize='compact', datatype='json'):
        self.url = 'https://www.alphavantage.co/query?function=' + self.function + '&symbol=' + self.symbol\
        + '&outputsize=' + outputsize + '&apikey=' + self.apikey + '&datatype=' + datatype

# DAILY_ADJUSTED API is now a Premium API
    def daily_adj(self, outputsize='compact', datatype='json'):
        self.url = 'https://www.alphavantage.co/query?function=' + self.function + '&symbol=' + self.symbol\
        + '&outputsize=' + outputsize + '&apikey=' + self.apikey + '&datatype=' + datatype
        
    def weekly(self, datatype='json'):
        self.url = 'https://www.alphavantage.co/query?function=' + self.function + '&symbol=' + self.symbol\
        + '&apikey=' + self.apikey + '&datatype=' + datatype
        
    def weekly_adj(self, datatype='json'):
        self.url = 'https://www.alphavantage.co/query?function=' + self.function + '&symbol=' + self.symbol\
        + '&apikey=' + self.apikey + '&datatype=' + datatype
    
    def monthly(self, datatype='json'):
        self.url = 'https://www.alphavantage.co/query?function=' + self.function + '&symbol=' + self.symbol\
        + '&apikey=' + self.apikey + '&datatype=' + datatype
        
    def monthly_adj(self, datatype='json'):
        self.url = 'https://www.alphavantage.co/query?function=' + self.function + '&symbol=' + self.symbol\
        + '&apikey=' + self.apikey + '&datatype=' + datatype

def api_call(function, symbol, **kwargs):
    construct = api_construct(function, symbol, key)
    
    if function == 'TIME_SERIES_INTRADAY':
        construct.intraday(**kwargs)
        tbl_name = 'intraday'
    elif function == 'TIME_SERIES_INTRADAY_EXTENDED':
        construct.intraday_ext(**kwargs)
        tbl_name = 'intraday_ext'
    elif function == 'TIME_SERIES_DAILY':
        construct.daily(**kwargs)
        tbl_name = 'daily'
    elif function == 'TIME_SERIES_DAILY_ADJUSTED':
        construct.daily_adj(**kwargs)
        tbl_name = 'daily_adj'
    elif function == 'TIME_SERIES_WEEKLY':
        construct.weekly(**kwargs)
        tbl_name = 'weekly'
    elif function == 'TIME_SERIES_WEEKLY_ADJUSTED':
        construct.weekly_adj(**kwargs)
        tbl_name = 'weekly_adj'
    elif function == 'TIME_SERIES_MONTHLY':
        construct.monthly(**kwargs)
        tbl_name = 'monthly'
    elif function == 'TIME_SERIES_MONTHLY_ADJUSTED':
        construct.monthly_adj(**kwargs)
        tbl_name = 'monthly_adj'
    else:
        print('Invalid function input')
        
    url = construct.url
    return url, tbl_name

## 4. Make the API Call 

In [18]:
x = api_call('TIME_SERIES_DAILY', 'NVDA')
url = x[0]
tbl_name = x[1]
r = requests.get(url)
print(f"Status code: {r.status_code}")  # Status Code 200 means the API call was successful
data = r.json()

Status code: 200


## 5.1 Clean and Transform Raw Data

In [19]:
tseries_keyname = list(data.keys())[1]

dicts, meta = data[tseries_keyname], data['Meta Data']

try:
    del interval  
except Exception:
    pass

try:
    symbol = meta['2. Symbol']
    interval = meta['4. Interval']
except KeyError:
    symbol = meta['2. Symbol']

values = []

In [20]:
tbl_keys = list( dicts[ list(dicts.keys())[0] ].keys() )

try:
    i = 0
    for date in dicts:
        values.append((f"{symbol}_{date}",symbol,date, interval))
        for key in tbl_keys:
            values[i] = values[i] + tuple( [float(dicts[date][key])] )  
        i += 1
except NameError:
    i = 0
    for date in dicts:
        values.append((f"{symbol}_{date}",symbol,date))
        for key in tbl_keys:
            values[i] = values[i] + tuple( [float(dicts[date][key])] )  
        i += 1

In [21]:
print(values[0][2])

2025-04-07


## 5.2 Make the ODBC Connection

In [22]:
db = []
with open('ODBC_Conn.txt') as file:
    for line in file:
        db.append(line.rstrip())

conn = pyodbc.connect(f'DRIVER={db[2]};\
                        SERVER={db[0]};\
                        DATABASE={db[1]};\
                        UID={db[3]};PWD={db[4]}')
cursor = conn.cursor()                

## 5.3 Insert the data into SQL Table

In [30]:
cursor.execute("DROP TABLE IF EXISTS StockData.dbo.##tempstock_tbl;")
try:
    interval
    cursor.execute("CREATE TABLE StockData.dbo.##tempstock_tbl\
        (stock_id VARCHAR(255), symbol VARCHAR(15), [date] DATETIME, interval VARCHAR(10));")
except NameError:
    cursor.execute("CREATE TABLE StockData.dbo.##tempstock_tbl\
        (stock_id VARCHAR(255), symbol VARCHAR(15), [date] DATE);")


for i in tbl_keys:
    colName = "[" + i + "]"    
    cursor.execute("EXEC StockData.dbo.addcol_tempstock @colName = ?",(colName))
    
headerStr = ''
for i in tbl_keys:
    colName = "[" + i + "]"
    headerStr = headerStr + colName + ','
headerStr = headerStr[:-1]

cursor.execute("SELECT COUNT(COLUMN_NAME) FROM StockData.INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME = '" + tbl_name + "';")
colSize = str(cursor.fetchone())
colSize = colSize.replace("(","")
colSize = colSize.replace(")","")
colSize = colSize.replace(",","")
colSize = int(colSize)
xValues = ""
for i in range(0, colSize):
    xValues = xValues + "?,"
xValues = xValues[:-1]    

try:
    interval
    cursor.fast_executemany = True
    cursor.executemany("INSERT INTO StockData.dbo.##tempstock_tbl (stock_id, symbol, [date], interval, " + headerStr + ")\
                  VALUES (" + xValues + ")",\
                  values)
except NameError:
    cursor.fast_executemany = True
    cursor.executemany("INSERT INTO StockData.dbo.##tempstock_tbl (stock_id, symbol, [date], " + headerStr + ")\
                  VALUES (" + xValues + ")",\
                  values)
    
cursor.execute("INSERT INTO StockData.dbo." + tbl_name + " SELECT * FROM StockData.dbo.##tempstock_tbl\
                WHERE stock_id NOT IN (SELECT stock_id FROM StockData.dbo." + tbl_name + ")")
conn.commit()